# Inspecting and cleaning the MAC data

In [2]:
# get macc data and imports
from acquire import get_mac_data
import pandas as pd
from prepare_module import summarize
import geopandas

In [10]:
# get mac data
df = get_mac_data()

## initial examination

In [ ]:
summarize(df)

#### Takeaways:
* nulls:
    * **CLOSEDDATE**: expected, implies that they ar not closed, will invesitgate to ensure there aren't any errors
    * **TOWNSHIP__C**: investigate why, may need some domain knowledge about indianapolis metro area townships, where they lie 
    * [indy gis township information](https://data.indy.gov/datasets/75f6558b7a074edb8fa5fa9b0d1fc867/explore)
    * [townships throughout the state](https://www.stats.indiana.edu/maptools/maps/boundary/townships_2000/townships_statewide.pdf)
    * **COUNCIL_DISTRICT__C**: investigate why, I think there may be a connection with the council districts nulls and the township nulls 
        * Note: there may be council redistricting and township redistrictinhg that may be impacting this 
    * **SOURCE_ID**: what is the source id in the first place? id it necessary for the analysis? What the the source Id correspond with? is each source id unique or are they connected to something meaningful?
    * **CITY__C**: The city can be found by plotting the address on a map.
        1. Find out if python has something for addresses in the united states
        1. If not, get the geometries for eact objectid
        1. Get the geometries for cities in indiana
        1. Find out where each objectid geometry falls and fill in the CITY__C nulls with the new data
* Features/Variables:
    * ORIGIN: see if there is a relationship between origin and the time a service case is handled
    * STATUS:
        * Question: What proportion of each *category* is open/closed?
        * Question: What proportion of each *city* is open/closed?
        * Question: What proportion of each *township* is open/closed?
        * Question: What proportion of each *council district* is open/closed?
        * Question: What proportion of each *zip code* is open/closed?
    * Dates:
        * CLOSEDDATE: 
            * What are some reasons for the closed dates having (over 100) on one second? Is there a system wide update that occurs?
            * Is there a connection between the createddate and the closeddate?
        * CREATEDDATE: 
            * Why are there mulitple CREATEDDATE entries with the same second? Are there multiple categories of service frequests oin one call that are all inputed? Are these duplicates?
    * COUNCIL_DISTRICT__C
        * See how many districts there are and which districts have the best response times and worst response times
    * ZIP__C
        1. Make sure this is the zip code
        1. Get a csv file with the zipcode geometries in the states of indiana
        1. Merge the csv file with this data using the zipcode as the point of merging
        1. Use this to create a map of who has good time, who has bad response time, and whjo has great response times
     * CITY__C : 
        1. Change indianapolisq mispelling
        1. Investigate the 'NONE'
        1. Determine which has the fastest turnaround for non-emergency calls
        1. Does the Mayor Action thing apply to the other cities??
    * INCIDENT_ADDRESS__C
        * See if we can plot the incident addresses and identify the city for each 
        * Check to see whay some incidetn addressess have so many counts int he data
    * SUBCATEGORY__C:
        * Consolidate subcategories
        * If after consolidation , place all low couont subcategories into a new subcaegory called 'low_count_subcategories'
    * KEYWORD__C:
        1. Order all the key words alphabetically to check for duplicateswith similar spelling, see if it is intentional
        1. Within duplicates, ensure the subcategories are similary to the duplucatedones
    * SOURCE_ID__C
        * not very clear how this works. is there a list of source_id signifieers or a key that shows the meaning and significance of this information?
    * CASENUMBER:
        * Whya re there duplicate case numbers?


#### Actions:
* nulls:
    * CLOSEDDATE: Initially, I will fill all the closeddates with a number far out into the future that is not possible to represent unclosed
    *  I will drop all nulls for now and later go and fill in the missing information with other data sources


## Addressing nulls and changing column names

In [9]:


# change columsn names to lower, remove the __c and that's it
col_rename = {}

for col in df.columns:
    col_rename[col] = col.lower().replace('__c', '')

df.rename(col_rename, axis=1, inplace = True)

In [70]:
# changing multiple columns to date time dtype
print(df.columns)

df[['createddate','lastmodifieddate', 'closeddate']] = df[['createddate','lastmodifieddate', 'closeddate']].apply(pd.to_datetime)

df.dtypes

Index(['source_id', 'keyword', 'subcategory', 'incident_address', 'township',
       'city', 'zip', 'council_district', 'createddate', 'lastmodifieddate',
       'closeddate', 'status', 'origin'],
      dtype='object')


ValueError: Cannot mix tz-aware with tz-naive values, at position 15

In [36]:
# changing the closeddate nulls to a date that hasnt come
df[df.closeddate.isna() == True]

df.closeddate.fillna('9999',  inplace=True)

In [41]:
df.closeddate.value_counts()

pd.to_datetime(df.closeddate, errors='coerce').value_counts()

closeddate
2018-05-01 16:32:02+00:00    200
2018-05-01 16:32:45+00:00    200
2015-11-09 12:26:20+00:00    188
2018-05-01 16:33:35+00:00    164
2018-05-01 16:32:55+00:00    142
                            ... 
2021-11-17 12:37:13+00:00      1
2021-12-06 17:07:37+00:00      1
2021-11-16 20:37:06+00:00      1
2021-11-29 19:57:33+00:00      1
2020-11-13 03:42:20+00:00      1
Name: count, Length: 883366, dtype: int64

In [49]:
df.closeddate[df.closeddate == '9999'] = '9999-12-31'

df.closeddate.value_counts()

# df.closeddate.apply(pd.to_datetime) An error

closeddate
9999-12-31                   35107
2018-05-01 16:32:45+00:00      200
2018-05-01 16:32:02+00:00      200
2015-11-09 12:26:20+00:00      188
2018-05-01 16:33:35+00:00      164
                             ...  
2021-11-17 12:37:13+00:00        1
2021-12-06 17:07:37+00:00        1
2021-11-16 20:37:06+00:00        1
2021-11-29 19:57:33+00:00        1
2020-11-13 03:42:20+00:00        1
Name: count, Length: 883367, dtype: int64

found a solution here [here](https://www.statology.org/pandas-out-of-bounds-nanosecond-timestamp/)

In [53]:
df.closeddate[df.closeddate == '9999-12-31'] = pd.Timestamp.max

df.closeddate.value_counts()

closeddate
2262-04-11 23:47:16.854775807    35107
2018-05-01 16:32:45+00:00          200
2018-05-01 16:32:02+00:00          200
2015-11-09 12:26:20+00:00          188
2018-05-01 16:33:35+00:00          164
                                 ...  
2021-11-17 12:37:13+00:00            1
2021-12-06 17:07:37+00:00            1
2021-11-16 20:37:06+00:00            1
2021-11-29 19:57:33+00:00            1
2020-11-13 03:42:20+00:00            1
Name: count, Length: 883367, dtype: int64

In [58]:
# dropping all other nulls in the ds
print(f'Dropping {df.shape[0] -  df.dropna().shape[0]} rows when dropping the nulls')

df.dropna(inplace=True)

df.shape

Dropping 24008 rows when dropping the nulls


### Investigating nulls

In [ ]:
# closseddate invesigation
close_date_null = df[df['closeddate'].isna() == True] # getting the df

# looking at the statuses, if the status is cklosed and there is no closeddate then there is an issue
close_date_null['status'].value_counts() # all of the nulls in this section are open which is to be expected



status
Open    35107
Name: count, dtype: int64

In [ ]:
# checking the township null segments
summarize(df[df['township'].isna()])

## Dropping uneccessary columns and adding the time delta

In [127]:
df = get_mac_data()

# change columsn names to lower, remove the __c and that's it
col_rename = {}

for col in df.columns:
    col_rename[col] = col.lower().replace('__c', '').replace('date', '')

col_rename['LASTMODIFIEDDATE'] = 'last_modified'

df.rename(col_rename, axis=1, inplace = True)

{'OBJECTID': 'objectid', 'CASENUMBER': 'casenumber', 'SOURCE_ID__C': 'source_id', 'KEYWORD__C': 'keyword', 'SUBCATEGORY__C': 'subcategory', 'INCIDENT_ADDRESS__C': 'incident_address', 'TOWNSHIP__C': 'township', 'CITY__C': 'city', 'ZIP__C': 'zip', 'COUNCIL_DISTRICT__C': 'council_district', 'CREATEDDATE': 'created', 'LASTMODIFIEDDATE': 'last_modified', 'CLOSEDDATE': 'closed', 'STATUS': 'status', 'ORIGIN': 'origin'}


In [116]:
# solution to out of bound and tz naive/tz aware discrepancy
pd.Timestamp.max.floor('30D').tz_localize('US/Eastern')

Timestamp('2262-03-31 00:00:00-0500', tz='US/Eastern')

In [119]:
# df[['createddate','lastmodifieddate', 'closeddate']] = df[['createddate','lastmodifieddate', 'closeddate']].apply(pd.to_datetime)

df['close_time'] = df['closeddate'] - df['createddate']

df.head()

,source_id,keyword,subcategory,incident_address,township,city,zip,council_district,createddate,lastmodifieddate,closeddate,status,origin,close_time
0,INV22-01609,Illegal Dumping and Junk/Trash,Trash Accumulation or Dumped Materials,1142 N GOODLET AVE,WAYNE,INDIANAPOLIS,46222,11.0,2022-01-31 15:25:28+00:00,2022-02-01 13:59:02+00:00,2022-02-01 13:59:02+00:00,Closed,RequestIndy Mobile,0 days 22:33:34
1,A22-375189,Animal,Abuse,3501 RALSTON AVE,CENTER,INDIANAPOLIS,46218,9.0,2022-01-28 18:35:54+00:00,2022-01-29 03:43:27+00:00,2022-01-29 03:43:20+00:00,Closed,Phone,0 days 09:07:26
2,108343054,Trash,96 Gal Cart - Stolen,1102 N PARKER AVE,CENTER,INDIANAPOLIS,46201,17.0,2022-01-28 21:12:49+00:00,2022-03-21 14:07:36+00:00,2022-03-21 14:07:36+00:00,Closed,Phone,51 days 16:54:47
3,Email Sent,Abandoned Vehicle,Street/Alley (ABV),4631 ROSSLYN AVE,WASHINGTON,INDIANAPOLIS,46205,9.0,2022-01-30 21:23:42+00:00,2022-02-03 21:29:59+00:00,2022-02-03 21:29:59+00:00,Closed,RequestIndy Online,4 days 00:06:17
4,108343404,Trash,96 Gal Trash (Missed),4360 SAWYER AVE,LAWRENCE,INDIANAPOLIS,46226,13.0,2022-01-31 15:44:58+00:00,2022-01-31 16:37:08+00:00,2022-01-31 16:37:08+00:00,Closed,RequestIndy Online,0 days 00:52:10


In [ ]:
summarize(df)

In [24]:
df['createddate'].

dtype('O')

## Geocoding the mac data using geopandas
> **Resources:**
> * [Guidance used](https://autogis-site.readthedocs.io/en/2019/notebooks/L3/geocoding_in_geopandas.html)
> * [Geopy Documentation](https://geopy.readthedocs.io/en/stable/#)


In [4]:
# getting the clean mac data
df = clean_mac_data()

Dropping 24008 rows


In [ ]:
# gtting only the addresses
print(df.columns)

# getting geocode format addresses
df['full_address'] = df['incident_address'] + ' ' +  df['city'] + ' ' + df['state'] + ' ' + df['zip'].astype(str)

df.head(1)

In [5]:
# getting geocode tool
from geopandas.tools import geocode

In [5]:
# getting a small batch to try it out first
geo = geocode(df.iloc[1]['full_address'], provider='Nominatim', user_agent='indy_action_time', timeout=10)
geo
# success, but now we need to set it up for all of them

,geometry,address
0,POINT (-86.13089 39.81994),"3501, Ralston Avenue, Indianapolis, Marion Cou..."


### Rate Limiter using geopy

In [6]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Point

In [7]:
# creating a smaller df to see if it works well
temp = pd.DataFrame(df.iloc[:5].loc[:,'full_address'])

In [8]:
# initiate geocoder 
geolocator = Nominatim(user_agent='indy_action_time')

# create a geopy rate limite:
geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# apply the geocoder with delay using the rate limiter
temp['temp'] = temp['full_address'].apply(geocode_with_delay)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('1142 N GOODLET AVE INDIANAPOLIS IN 46222',), **{}).
Traceback (most recent call last):
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/adapters.py", line 447, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/adapters.py", line 475, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 502

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/extra/rate_limiter.py", line 136, in _retries_gen
    yield i  # Run t

In [9]:
temp

,full_address,temp
0,1142 N GOODLET AVE INDIANAPOLIS IN 46222,None
1,3501 RALSTON AVE INDIANAPOLIS IN 46218,None
2,1102 N PARKER AVE INDIANAPOLIS IN 46201,None
3,4631 ROSSLYN AVE INDIANAPOLIS IN 46205,None
4,4360 SAWYER AVE INDIANAPOLIS IN 46226,None


# Creating the function

In [3]:
def clean_mac_data():
    '''
    This function acquires mac data, renames columns, changes dtypes, fills in null values for the closed date, drops all other null values,
    drops objectid, caseneumber, and source_id, and creates a new columns
    Modules:
        from acquire import get_mac_data
        import pandas as pd
    '''
    df = get_mac_data()

    # change columsn names to lower, remove the __c and that's it
    col_rename = {}

    for col in df.columns:
        col_rename[col] = col.lower().replace('__c', '').replace('date', '')

    col_rename['LASTMODIFIEDDATE'] = 'last_modified'

    df.rename(col_rename, axis=1, inplace = True)

    # changing multiple columns to date time dtype
    df[['created','last_modified', 'closed']] = df[['created','last_modified', 'closed']].apply(pd.to_datetime)

    # filling in the closeddate nulls with the date the furthest out
    df.closed.fillna(pd.Timestamp.max.floor('30D').tz_localize('US/Eastern'), inplace=True)

    # dropping null values
    print(f'Dropping {df.shape[0] - df.dropna().shape[0]} rows')
    df.dropna(inplace=True)

    # dropping the unneeded columns
    df.drop(['objectid', 'casenumber', 'source_id'], axis=1, inplace=True)

    # adding the time from creation to closing
    df['close_time'] = df['closed'] - df['created']

    # adding the state
    df['state'] = 'IN'

    # getting geocode format addresses
    df['full_address'] = df['incident_address'] + ' ' +  df['city'] + ' ' + df['state'] + ' ' + df['zip'].astype(str)

    return df

In [4]:
df = clean_mac_data()

df

Dropping 24008 rows


,keyword,subcategory,incident_address,township,city,zip,council_district,created,last_modified,closed,status,origin,close_time,state,full_address
0,Illegal Dumping and Junk/Trash,Trash Accumulation or Dumped Materials,1142 N GOODLET AVE,WAYNE,INDIANAPOLIS,46222,11.0,2022-01-31 15:25:28+00:00,2022-02-01 13:59:02+00:00,2022-02-01 13:59:02+00:00,Closed,RequestIndy Mobile,0 days 22:33:34,IN,1142 N GOODLET AVE INDIANAPOLIS IN 46222
1,Animal,Abuse,3501 RALSTON AVE,CENTER,INDIANAPOLIS,46218,9.0,2022-01-28 18:35:54+00:00,2022-01-29 03:43:27+00:00,2022-01-29 03:43:20+00:00,Closed,Phone,0 days 09:07:26,IN,3501 RALSTON AVE INDIANAPOLIS IN 46218
2,Trash,96 Gal Cart - Stolen,1102 N PARKER AVE,CENTER,INDIANAPOLIS,46201,17.0,2022-01-28 21:12:49+00:00,2022-03-21 14:07:36+00:00,2022-03-21 14:07:36+00:00,Closed,Phone,51 days 16:54:47,IN,1102 N PARKER AVE INDIANAPOLIS IN 46201
3,Abandoned Vehicle,Street/Alley (ABV),4631 ROSSLYN AVE,WASHINGTON,INDIANAPOLIS,46205,9.0,2022-01-30 21:23:42+00:00,2022-02-03 21:29:59+00:00,2022-02-03 21:29:59+00:00,Closed,RequestIndy Online,4 days 00:06:17,IN,4631 ROSSLYN AVE INDIANAPOLIS IN 46205
4,Trash,96 Gal Trash (Missed),4360 SAWYER AVE,LAWRENCE,INDIANAPOLIS,46226,13.0,2022-01-31 15:44:58+00:00,2022-01-31 16:37:08+00:00,2022-01-31 16:37:08+00:00,Closed,RequestIndy Online,0 days 00:52:10,IN,4360 SAWYER AVE INDIANAPOLIS IN 46226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940633,Animal,Noise (Animal),3602 E VERMONT ST,CENTER,INDIANAPOLIS,46201,12.0,2020-10-28 13:41:38+00:00,2020-10-28 17:43:21+00:00,2020-10-28 17:43:16+00:00,Closed,Phone,0 days 04:01:38,IN,3602 E VERMONT ST INDIANAPOLIS IN 46201
940634,Chuckhole,Street (Chuckhole),5069 W SOUTHPORT RD,DECATUR,INDIANAPOLIS,46221,20.0,2022-03-02 18:04:11+00:00,2022-03-09 19:08:30+00:00,2022-03-09 19:08:30+00:00,Closed,Phone,7 days 01:04:19,IN,5069 W SOUTHPORT RD INDIANAPOLIS IN 46221
940635,Weeds,Private Property (Complaint),625 N TUXEDO ST,CENTER,INDIANAPOLIS,46201,12.0,2021-06-23 15:58:24+00:00,2021-06-24 18:44:11+00:00,2021-06-24 18:44:11+00:00,Closed,Phone,1 days 02:45:47,IN,625 N TUXEDO ST INDIANAPOLIS IN 46201
940636,Debris/Litter,Debris in Alley/Street,2801 N MERIDIAN ST,CENTER,INDIANAPOLIS,46208,9.0,2020-10-29 10:10:10+00:00,2020-12-03 20:47:07+00:00,2020-10-29 18:12:11+00:00,Closed,RequestIndy Mobile,0 days 08:02:01,IN,2801 N MERIDIAN ST INDIANAPOLIS IN 46208


#### Notes:
* STATUS: is it useful to use the open status information? I don't think so

#### Action:
* Removing all open status claims
    * The open status claims all have no closeddate which means we cannot use this data to train our model on. 
    * This data can be useful for the identifyinf which type of service requests are not handled for extedned periods of time, but that should be addressed in the the data already available unless there are some requests that have never closed even when introduced previosuly. That seems like a special case experience whoc deserves its own investigation